In [88]:
import sys 
sys.executable

'd:\\emi_prediction_app\\venv\\Scripts\\python.exe'

In [90]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.pyfunc


In [91]:
data = pd.read_csv(r"D:\emi_prediction_app\emi_prediction_dataset.csv",nrows=1000)

# data=data.head(200000)


In [92]:
data.head()

,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38.0,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38.0,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38.0,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58.0,Female,Married,High School,66800.0,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
4,48.0,Female,Married,Professional,57300.0,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0


In [93]:
df=pd.DataFrame(data)

In [94]:
df.shape

(1000, 27)

In [95]:
df.describe()

,age,years_of_employment,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,current_emi_amount,credit_score,emergency_fund,requested_amount,requested_tenure,max_monthly_emi
count,1000.00000,1000.000000,992.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,994.000000,995.000000,1.000000e+03,1000.00000,1000.000000
mean,38.84700,5.558100,6273.893145,2.975000,1.975000,4660.300000,4102.100000,5785.600000,12889.900000,7103.100000,4516.300000,700.225352,99975.577889,3.624710e+05,28.91200,6828.868390
std,9.09331,6.570005,8912.697937,1.095251,1.095251,5044.317501,7234.318278,3408.907933,6873.908539,4438.414121,6951.161267,82.882119,86497.513232,3.453920e+05,17.99984,7768.584568
min,27.00000,0.500000,0.000000,1.000000,0.000000,0.000000,0.000000,900.000000,2200.000000,700.000000,0.000000,0.000000,3400.000000,1.000000e+04,3.00000,500.000000
25%,32.00000,1.100000,0.000000,2.000000,1.000000,0.000000,0.000000,3200.000000,7600.000000,3800.000000,0.000000,654.000000,38550.000000,1.227500e+05,15.00000,500.000000
50%,38.00000,3.100000,0.000000,3.000000,2.000000,3150.000000,0.000000,5100.000000,11700.000000,6200.000000,0.000000,701.000000,75300.000000,2.130000e+05,24.00000,4375.000000
75%,48.00000,7.300000,11925.000000,4.000000,3.000000,9025.000000,6925.000000,7525.000000,16800.000000,9225.000000,7825.000000,747.000000,132100.000000,4.740000e+05,40.00000,9720.000000
max,58.00000,36.000000,51500.000000,5.000000,4.000000,15000.000000,24900.000000,23300.000000,42700.000000,29100.000000,36700.000000,1200.000000,579000.000000,1.489000e+06,84.00000,73986.000000


In [96]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     1000 non-null   float64
 1   gender                  1000 non-null   object 
 2   marital_status          1000 non-null   object 
 3   education               993 non-null    object 
 4   monthly_salary          1000 non-null   object 
 5   employment_type         1000 non-null   object 
 6   years_of_employment     1000 non-null   float64
 7   company_type            1000 non-null   object 
 8   house_type              1000 non-null   object 
 9   monthly_rent            992 non-null    float64
 10  family_size             1000 non-null   int64  
 11  dependents              1000 non-null   int64  
 12  school_fees             1000 non-null   float64
 13  college_fees            1000 non-null   float64
 14  travel_expenses         1000 non-null   f

In [97]:
numerical_cols = []
categorical_cols = []

for col in df.columns:
    if data[col].dtype in ['int64', 'float64']:
        numerical_cols.append(col)
    else:
        categorical_cols.append(col)



In [98]:
for col in categorical_cols:
    print(col,' ',df[col].unique(),' ',df[col].dtype)
    print()

gender   ['Female' 'Male' 'female' 'male' 'M' 'MALE' 'F' 'FEMALE']   object

marital_status   ['Married' 'Single']   object

education   ['Professional' 'Graduate' 'High School' 'Post Graduate' nan]   object

monthly_salary   ['82600.0' '21500.0' '86100.0' '66800.0' '57300.0' '38800.0' '27100.0'
 '392044.0' '47700.0' '129200.0' '58600.0' '47000.0' '11837.0' '53900.0'
 '110800.0' '64000.0' '46600.0' '77400.0' '40700.0' '47500.0' '65600.0'
 '67300.0' '15000.0' '29200.0' '61600.0' '72800.0' '95000.0' '123200.0'
 '28400.0' '29800.0' '32600.0' '55200.0' '120900.0' '28900.0' '148600.0'
 '37600.0' '53700.0' '89500.0' '28500.0' '49100.0' '60400.0' '43500.0'
 '77300.0' '61000.0' '57800.0' '31000.0' '158100.0' '24000.0' '93257.0'
 '103200.0' '17700.0' '99100.0' '66300.0' '118500.0' '60100.0' '40100.0'
 '92200.0' '400172.0' '43200.0' '45700.0' '54900.0' '319242.0' '103400.0'
 '64400.0' '34800.0' '49700.0' '60900.0' '70400.0' '61300.0' '107500.0'
 '35300.0' '98800.0' '47200.0' '26700.0' '43600.0' 

In [99]:
for col in ['monthly_salary','bank_balance']:
    if col in categorical_cols:
        categorical_cols.remove(col)
        numerical_cols.append(col)


In [100]:
for col in numerical_cols:
    print(col,' ',df[col].unique(),' ',df[col].dtype)
    print()

age   [38. 58. 48. 32. 27. 47. 37.]   float64

years_of_employment   [ 0.9  7.   5.8  2.2  3.4  5.  36.  12.9  1.9  7.5  3.7  1.3  3.2  0.5
 26.   0.8  4.2  2.6  3.9  9.7  4.7  9.6  1.7  9.4  4.1  1.   7.6 16.
  2.5  1.6 17.1  6.6 12.6  2.3  4.5 11.3 10.   0.7  3.   2.1  2.7  0.6
  8.6  5.9  8.4 12.4  2.8 15.9 10.2  1.2  1.5 14.   1.1  6.5  7.4  7.2
 20.7 17.6 16.6 10.9 20.6 29.7  8.5 10.8 14.2  9.9 24.3  9.1 11.4 11.8
  2.   6.2  6.9  5.2 11.7 18.1  8.2  3.1  3.3  7.1  5.5 10.4  4.   2.4
  8.3  6.1  4.4  8.8  5.6  5.4  4.6 17.8 20.2 22.7 13.6  8.   4.3  2.9
 10.5 13.1 14.1 11.2  4.8 15.7  6.3  5.3 18.9 11.9 21.3  6.4 15.5  3.5
 12.   1.8 13.3  1.4 20.5  8.9 10.3  8.1 14.8 32.   5.1  7.9 21.9  7.7
 15.  25.8 14.3  6.7  3.8 21.8  9.2  7.3  5.7  4.9  9.  15.1  6.8 15.3
 13.  20.  12.2 12.1 13.8 23.8 12.8 27.7  6.  18.2 15.6  8.7 17.4  3.6
 17.3  9.5 33.4 11.1 15.4 20.1 24.1 10.1 19.7 16.5 10.7 23.6 13.9  9.8
  9.3 25.3 14.4 21.4]   float64

monthly_rent   [20000.     0. 13500. 13100. 117

In [101]:
for col in numerical_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)')
    df[col] = df[col].astype(float)
    df[col].fillna(df[col].mean(), inplace=True)


C:\Users\Vishal Singla\AppData\Local\Temp\ipykernel_18244\4264373797.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\Vishal Singla\AppData\Local\Temp\ipykernel_18244\4264373797.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [102]:
for col in numerical_cols:
    print(col,' ',df[col].isnull().sum())


age   0
years_of_employment   0
monthly_rent   0
family_size   0
dependents   0
school_fees   0
college_fees   0
travel_expenses   0
groceries_utilities   0
other_monthly_expenses   0
current_emi_amount   0
credit_score   0
emergency_fund   0
requested_amount   0
requested_tenure   0
max_monthly_emi   0
monthly_salary   0
bank_balance   0


In [103]:
for col in categorical_cols:
    print(col,' ',df[col].isnull().sum())

gender   0
marital_status   0
education   7
employment_type   0
company_type   0
house_type   0
existing_loans   0
emi_scenario   0
emi_eligibility   0


In [104]:
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


C:\Users\Vishal Singla\AppData\Local\Temp\ipykernel_18244\54059043.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [105]:
for col in categorical_cols:
    print(col,' ',df[col].isnull().sum())

gender   0
marital_status   0
education   0
employment_type   0
company_type   0
house_type   0
existing_loans   0
emi_scenario   0
emi_eligibility   0


In [106]:
df['gender'].unique()

array(['Female', 'Male', 'female', 'male', 'M', 'MALE', 'F', 'FEMALE'],
      dtype=object)

In [107]:
df['gender']=df['gender'].map({'Female':'F','Male':'M','female':'F','male':'M','MALE':'M','F':'F','FEMALE':'F'})

In [108]:
df['gender'].unique()

array(['F', 'M', nan], dtype=object)

In [110]:
df['gender'].fillna(df['gender'].mode()[0],inplace=True)

In [111]:
df['gender'].unique()

array(['F', 'M'], dtype=object)

In [112]:
df.isnull().sum()

age                       0
gender                    0
marital_status            0
education                 0
monthly_salary            0
employment_type           0
years_of_employment       0
company_type              0
house_type                0
monthly_rent              0
family_size               0
dependents                0
school_fees               0
college_fees              0
travel_expenses           0
groceries_utilities       0
other_monthly_expenses    0
existing_loans            0
current_emi_amount        0
credit_score              0
bank_balance              0
emergency_fund            0
emi_scenario              0
requested_amount          0
requested_tenure          0
emi_eligibility           0
max_monthly_emi           0
dtype: int64

In [113]:
for col in categorical_cols:
    print(col,' ',df[col].unique())

gender   ['F' 'M']
marital_status   ['Married' 'Single']
education   ['Professional' 'Graduate' 'High School' 'Post Graduate']
employment_type   ['Private' 'Government' 'Self-employed']
company_type   ['Mid-size' 'MNC' 'Startup' 'Large Indian' 'Small']
house_type   ['Rented' 'Family' 'Own']
existing_loans   ['Yes' 'No']
emi_scenario   ['Personal Loan EMI' 'E-commerce Shopping EMI' 'Education EMI'
 'Vehicle EMI' 'Home Appliances EMI']
emi_eligibility   ['Not_Eligible' 'Eligible' 'High_Risk']


In [114]:
education_order = ['High School','Graduate','Post Graduate','Professional']
oe = OrdinalEncoder(categories=[education_order])
df['education'] = oe.fit_transform(df[['education']])


In [115]:
df['education'].unique()

array([3., 1., 0., 2.])

In [116]:
nominal_cols = categorical_cols.copy()
nominal_cols.remove('education')

df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)
df=df.astype('float')


In [117]:
for col in df.columns:
    print(col,' ',df[col].unique(),' ',df[col].dtype,'\n')

age   [38. 58. 48. 32. 27. 47. 37.]   float64 

education   [3. 1. 0. 2.]   float64 

monthly_salary   [ 82600.  21500.  86100.  66800.  57300.  38800.  27100. 392044.  47700.
 129200.  58600.  47000.  11837.  53900. 110800.  64000.  46600.  77400.
  40700.  47500.  65600.  67300.  15000.  29200.  61600.  72800.  95000.
 123200.  28400.  29800.  32600.  55200. 120900.  28900. 148600.  37600.
  53700.  89500.  28500.  49100.  60400.  43500.  77300.  61000.  57800.
  31000. 158100.  24000.  93257. 103200.  17700.  99100.  66300. 118500.
  60100.  40100.  92200. 400172.  43200.  45700.  54900. 319242. 103400.
  64400.  34800.  49700.  60900.  70400.  61300. 107500.  35300.  98800.
  47200.  26700.  43600.  81600.  37300.  49300.  56600.  33100.  38900.
  68600. 118100.  40800.  93200.  32000.  69100. 125200.  23600.  23300.
 113300.  44200. 166700.  40500.  77900.  74600.  14400.  57900. 103000.
  52300.  63000.  46100.  39700.  18300.  29500.  66500.  20000.  42900.
 105400. 127300.  296

In [118]:
def classify(row):
    if row['emi_eligibility_High_Risk'] == 1:
        return 'High_Risk'
    elif row['emi_eligibility_Not_Eligible'] == 1:
        return 'Not_Eligible'
    else:
        return 'Eligible'

df['emi_eligibility'] = df.apply(classify, axis=1)


In [120]:
for col in df.columns:
    print(col,' ',df[col].unique(),' ',df[col].dtype,'\n')

age   [38. 58. 48. 32. 27. 47. 37.]   float64 

education   [3. 1. 0. 2.]   float64 

monthly_salary   [ 82600.  21500.  86100.  66800.  57300.  38800.  27100. 392044.  47700.
 129200.  58600.  47000.  11837.  53900. 110800.  64000.  46600.  77400.
  40700.  47500.  65600.  67300.  15000.  29200.  61600.  72800.  95000.
 123200.  28400.  29800.  32600.  55200. 120900.  28900. 148600.  37600.
  53700.  89500.  28500.  49100.  60400.  43500.  77300.  61000.  57800.
  31000. 158100.  24000.  93257. 103200.  17700.  99100.  66300. 118500.
  60100.  40100.  92200. 400172.  43200.  45700.  54900. 319242. 103400.
  64400.  34800.  49700.  60900.  70400.  61300. 107500.  35300.  98800.
  47200.  26700.  43600.  81600.  37300.  49300.  56600.  33100.  38900.
  68600. 118100.  40800.  93200.  32000.  69100. 125200.  23600.  23300.
 113300.  44200. 166700.  40500.  77900.  74600.  14400.  57900. 103000.
  52300.  63000.  46100.  39700.  18300.  29500.  66500.  20000.  42900.
 105400. 127300.  296

In [123]:
y_class = df['emi_eligibility']
y_reg = df['max_monthly_emi']

X = df.drop(columns=['emi_eligibility','emi_eligibility_High_Risk',
                       'emi_eligibility_Not_Eligible','max_monthly_emi'])


In [124]:
X.columns

Index(['age', 'education', 'monthly_salary', 'years_of_employment',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'current_emi_amount', 'credit_score',
       'bank_balance', 'emergency_fund', 'requested_amount',
       'requested_tenure', 'gender_M', 'marital_status_Single',
       'employment_type_Private', 'employment_type_Self-employed',
       'company_type_MNC', 'company_type_Mid-size', 'company_type_Small',
       'company_type_Startup', 'house_type_Own', 'house_type_Rented',
       'existing_loans_Yes', 'emi_scenario_Education EMI',
       'emi_scenario_Home Appliances EMI', 'emi_scenario_Personal Loan EMI',
       'emi_scenario_Vehicle EMI'],
      dtype='object')

In [125]:
y_class

0      Not_Eligible
1      Not_Eligible
2          Eligible
3          Eligible
4      Not_Eligible
           ...     
995    Not_Eligible
996        Eligible
997    Not_Eligible
998    Not_Eligible
999       High_Risk
Name: emi_eligibility, Length: 1000, dtype: object

In [126]:
y_reg

0        500.0
1        700.0
2      27775.0
3      16170.0
4        500.0
        ...   
995      500.0
996    13840.0
997     4256.0
998      500.0
999     8040.0
Name: max_monthly_emi, Length: 1000, dtype: float64

In [127]:
df.head()

,age,education,monthly_salary,years_of_employment,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,...,house_type_Own,house_type_Rented,existing_loans_Yes,emi_scenario_Education EMI,emi_scenario_Home Appliances EMI,emi_scenario_Personal Loan EMI,emi_scenario_Vehicle EMI,emi_eligibility_High_Risk,emi_eligibility_Not_Eligible,emi_eligibility
0,38.0,3.0,82600.0,0.9,20000.0,3.0,2.0,0.0,0.0,7200.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,Not_Eligible
1,38.0,1.0,21500.0,7.0,0.0,2.0,1.0,5100.0,0.0,1400.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Not_Eligible
2,38.0,3.0,86100.0,5.8,0.0,4.0,3.0,0.0,0.0,10200.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Eligible
3,58.0,0.0,66800.0,2.2,0.0,5.0,4.0,11400.0,0.0,6200.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Eligible
4,48.0,3.0,57300.0,3.4,0.0,4.0,3.0,9400.0,21300.0,3600.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,Not_Eligible


In [128]:
df.columns

Index(['age', 'education', 'monthly_salary', 'years_of_employment',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'current_emi_amount', 'credit_score',
       'bank_balance', 'emergency_fund', 'requested_amount',
       'requested_tenure', 'max_monthly_emi', 'gender_M',
       'marital_status_Single', 'employment_type_Private',
       'employment_type_Self-employed', 'company_type_MNC',
       'company_type_Mid-size', 'company_type_Small', 'company_type_Startup',
       'house_type_Own', 'house_type_Rented', 'existing_loans_Yes',
       'emi_scenario_Education EMI', 'emi_scenario_Home Appliances EMI',
       'emi_scenario_Personal Loan EMI', 'emi_scenario_Vehicle EMI',
       'emi_eligibility_High_Risk', 'emi_eligibility_Not_Eligible',
       'emi_eligibility'],
      dtype='object')

Classification

In [129]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_class_encoded = le.fit_transform(y_class)


In [130]:
y_class

0      Not_Eligible
1      Not_Eligible
2          Eligible
3          Eligible
4      Not_Eligible
           ...     
995    Not_Eligible
996        Eligible
997    Not_Eligible
998    Not_Eligible
999       High_Risk
Name: emi_eligibility, Length: 1000, dtype: object

In [132]:
#0-'Eligible',1-'High-Risk',2-'Not-Eligible'

In [133]:
y_class_encoded

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 1, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0,

In [134]:
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y_class_encoded)


Classification

In [135]:
X_train_c, X_test_c, y_train_c, y_test_c =train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [137]:
X_train_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1864 entries, 1113 to 860
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   age                               1864 non-null   float64
 1   education                         1864 non-null   float64
 2   monthly_salary                    1864 non-null   float64
 3   years_of_employment               1864 non-null   float64
 4   monthly_rent                      1864 non-null   float64
 5   family_size                       1864 non-null   float64
 6   dependents                        1864 non-null   float64
 7   school_fees                       1864 non-null   float64
 8   college_fees                      1864 non-null   float64
 9   travel_expenses                   1864 non-null   float64
 10  groceries_utilities               1864 non-null   float64
 11  other_monthly_expenses            1864 non-null   float64
 12  current_e

In [138]:
numerical_cols_c = [col for col in X_train_c.columns if X_train_c[col].dtype == 'float64']


Regression

In [140]:
X_train_r, X_test_r, y_train_r, y_test_r =train_test_split(X, y_reg, test_size=0.2, random_state=42)


In [141]:
X_train_r.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 29 to 102
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   age                               800 non-null    float64
 1   education                         800 non-null    float64
 2   monthly_salary                    800 non-null    float64
 3   years_of_employment               800 non-null    float64
 4   monthly_rent                      800 non-null    float64
 5   family_size                       800 non-null    float64
 6   dependents                        800 non-null    float64
 7   school_fees                       800 non-null    float64
 8   college_fees                      800 non-null    float64
 9   travel_expenses                   800 non-null    float64
 10  groceries_utilities               800 non-null    float64
 11  other_monthly_expenses            800 non-null    float64
 12  current_emi_

In [142]:
numerical_cols_r = [col for col in X_train_r.columns if X_train_r[col].dtype == 'float64']


In [23]:
# scaling_col=[]
# for col in X.columns:
#     if X[col].dtype=='float64':
#         scaling_col.append(col)

In [143]:
scaler_c = StandardScaler()
X_train_c[numerical_cols_c] = scaler_c.fit_transform(X_train_c[numerical_cols_c])
X_test_c[numerical_cols_c] = scaler_c.transform(X_test_c[numerical_cols_c])

In [144]:
scaler_r = StandardScaler()
X_train_r[numerical_cols_r] = scaler_r.fit_transform(X_train_r[numerical_cols_r])
X_test_r[numerical_cols_r] = scaler_r.transform(X_test_r[numerical_cols_r])


In [151]:
! pip install mlflow

In [150]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error,mean_absolute_error
import mlflow.sklearn

In [166]:
mlflow.set_tracking_uri('file:///D:/emi_prediction_app/mlruns_new')
mlflow.set_experiment('Classfication_Models')

<Experiment: artifact_location='file:///D:/emi_prediction_app/mlruns_new/252743254839674411', creation_time=1767692223926, experiment_id='252743254839674411', last_update_time=1767692223926, lifecycle_stage='active', name='Classfication_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [161]:
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: file:///D:/emi_prediction_app/mlruns_new


In [162]:
# ============================
# Classification MODEL
# ============================
mlflow.end_run()
# ----- LOGISTIC REGRESSION -----
with mlflow.start_run(run_name="Logistic_Regression"):
    log_clf = LogisticRegression(max_iter=500)
    log_clf.fit(X_train_c, y_train_c)
    y_pred = log_clf.predict(X_test_c)
    acc=accuracy_score(y_test_c, y_pred)
    f1=f1_score(y_test_c, y_pred, average='weighted')
    rms=np.sqrt(mean_squared_error(y_test_c, y_pred))
    print("rms:",rms)
    print("Accuracy:",acc)
    print("F1-score:",f1)
    mlflow.log_metric("accuracy", accuracy_score(y_test_c, y_pred))
    mlflow.log_metric("f1", f1_score(y_test_c, y_pred, average='weighted'))
    mlflow.sklearn.log_model(log_clf, "logistic_model", registered_model_name="Logistic_Classification_Model")


2026/01/06 15:12:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


rms: 0.42411262695493096
Accuracy: 0.8972162740899358
F1-score: 0.897343084203729


2026/01/06 15:12:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Logistic_Classification_Model'.
Created version '1' of model 'Logistic_Classification_Model'.


In [163]:
mlflow.end_run()
with mlflow.start_run(run_name="XGB_Classifier"):
    xgb_clf = XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='mlogloss')
    xgb_clf.fit(X_train_c, y_train_c)
    y_pred = xgb_clf.predict(X_test_c)
    acc= accuracy_score(y_test_c, y_pred)
    f1= f1_score(y_test_c, y_pred, average='weighted')
    print("Accuracy:",acc)
    print("F1-score:",f1)
    mlflow.log_metric("accuracy", accuracy_score(y_test_c, y_pred))
    mlflow.log_metric("f1", f1_score(y_test_c, y_pred, average='weighted'))
    mlflow.sklearn.log_model(xgb_clf, "xgb_classifier_model", registered_model_name="XGB_Classification_Model")


d:\emi_prediction_app\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [15:13:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2026/01/06 15:13:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Accuracy: 0.9785867237687366
F1-score: 0.9786427869919522


2026/01/06 15:14:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGB_Classification_Model'.
Created version '1' of model 'XGB_Classification_Model'.


In [167]:


from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
mlflow.end_run()
# mlflow.set_experiment("emi_prediction_classification1")

with mlflow.start_run(run_name="RF_Classifier"):

    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train_c, y_train_c)

    y_pred_c = clf.predict(X_test_c)

    acc = accuracy_score(y_test_c, y_pred_c)
    f1 = f1_score(y_test_c, y_pred_c, average='weighted')

    print("Accuracy:", acc)
    print("F1 Score:", f1)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)

    # Save model
    mlflow.sklearn.log_model(
        clf,
        "classification_model",
        registered_model_name="EMI_Classification_Model"
    )


2026/01/06 15:16:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Accuracy: 0.9593147751605996
F1 Score: 0.9594151585026859


2026/01/06 15:16:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'EMI_Classification_Model' already exists. Creating a new version of this model...
Created version '3' of model 'EMI_Classification_Model'.


In [168]:
import joblib

In [169]:
joblib.dump(clf,"clf_model.pkl")
print("clf model pickled")

clf model pickled


In [171]:
# ============================
# Regression MODEL
# ============================

from sklearn.ensemble import RandomForestRegressor
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Regression_Models")

with mlflow.start_run(run_name="RF_Regression"):

    reg = RandomForestRegressor(n_estimators=200, random_state=42)
    reg.fit(X_train_r, y_train_r)

    y_pred_r = reg.predict(X_test_r)

    r2 = r2_score(y_test_r, y_pred_r)
    mae = mean_absolute_error(y_test_r, y_pred_r)

    print("R2 Score:", r2)
    print("MAE:", mae)

    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mae", mae)

    # Save regression model
    mlflow.sklearn.log_model(
        reg,
        "regression_model",
        registered_model_name="EMI_Regression_Model"
    )


2026/01/06 15:18:45 INFO mlflow.tracking.fluent: Experiment with name 'Regression_Models' does not exist. Creating a new experiment.


2026/01/06 15:18:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


R2 Score: 0.7943008581890143
MAE: 2160.67192525


2026/01/06 15:18:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'EMI_Regression_Model' already exists. Creating a new version of this model...
Created version '2' of model 'EMI_Regression_Model'.


In [172]:
mlflow.end_run()
with mlflow.start_run(run_name="Linear_Regression"):
    lin_reg = LinearRegression()
    lin_reg.fit(X_train_r, y_train_r)
    y_pred = lin_reg.predict(X_test_r)
    print("r2", r2_score(y_test_r, y_pred))
    print("mae", mean_absolute_error(y_test_r, y_pred))
    mlflow.log_metric("r2", r2_score(y_test_r, y_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_test_r, y_pred))
    mlflow.sklearn.log_model(lin_reg, "linear_model", registered_model_name="Linear_Regression_Model")


2026/01/06 15:19:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


r2 0.6846204850563589
mae 2964.562639571011


2026/01/06 15:19:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Linear_Regression_Model'.
Created version '1' of model 'Linear_Regression_Model'.


In [173]:
mlflow.end_run()
with mlflow.start_run(run_name="XGB_Regressor"):
    xgb_reg = XGBRegressor(n_estimators=200)
    xgb_reg.fit(X_train_r, y_train_r)
    y_pred = xgb_reg.predict(X_test_r)
    print("r2", r2_score(y_test_r, y_pred))
    print("mae", mean_absolute_error(y_test_r, y_pred))
    mlflow.log_metric("r2", r2_score(y_test_r, y_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_test_r, y_pred))
    mlflow.sklearn.log_model(xgb_reg, "xgb_regressor_model", registered_model_name="XGB_Regression_Model")

2026/01/06 15:19:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


r2 0.8423338453639665
mae 1866.305760833931


2026/01/06 15:19:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGB_Regression_Model'.
Created version '1' of model 'XGB_Regression_Model'.


In [174]:
joblib.dump(xgb_reg, "reg_model.pkl")

['reg_model.pkl']

In [175]:
X.columns

Index(['age', 'education', 'monthly_salary', 'years_of_employment',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'current_emi_amount', 'credit_score',
       'bank_balance', 'emergency_fund', 'requested_amount',
       'requested_tenure', 'gender_M', 'marital_status_Single',
       'employment_type_Private', 'employment_type_Self-employed',
       'company_type_MNC', 'company_type_Mid-size', 'company_type_Small',
       'company_type_Startup', 'house_type_Own', 'house_type_Rented',
       'existing_loans_Yes', 'emi_scenario_Education EMI',
       'emi_scenario_Home Appliances EMI', 'emi_scenario_Personal Loan EMI',
       'emi_scenario_Vehicle EMI'],
      dtype='object')

In [37]:
# import os
# print(os.listdir())

In [38]:
# print(clf)

In [39]:
# print(reg)

In [40]:
# import streamlit as st
# import pandas as pd
# import numpy as np
# import mlflow.pyfunc

# st.title("EMI Eligibility Prediction App")

# # Load both models
# clf_model = mlflow.pyfunc.load_model("models:/EMI_Classification_Model/1")
# # reg_model = mlflow.pyfunc.load_model("runs:/058c8021ff0c47c7ae9d06ca92a0defd/regression_model")

# # User input form
# age = st.number_input("Age", 18, 80)
# salary = st.number_input("Monthly Salary", 1000, 500000)
# bank_balance = st.number_input("Bank Balance", 0, 10000000)
# education = st.selectbox("Education", ["High School", "Graduate", "Post Graduate", "Professional"])
# gender = st.selectbox("Gender", ["M", "F"])

# # Convert manually (same as training!)
# edu_map = {'High School':0, 'Graduate':1, 'Post Graduate':2, 'Professional':3}

# input_dict = {
#     "age": age,
#     "monthly_salary": salary,
#     "bank_balance": bank_balance,
#     "education": edu_map[education],
#     "gender_M": 1 if gender=="M" else 0
# }

# # Convert to DataFrame
# df = pd.DataFrame([input_dict])

# if st.button("Check Eligibility"):
#     pred_class = clf_model.predict(df)[0]
#     # pred_emi = reg_model.predict(df)[0]

#     st.subheader("Prediction Result:")
#     st.write("**Eligibility:**", pred_class)
#     # st.write("**Max EMI Allowed:** ₹", int(pred_emi))
